# Create Agent with Function Definition

In this notebook we will create an Agent for Amazon Bedrock using the new capabilities for function definition.

We will use the HR agent as example. With this agent, you can check your available vacation days and request a new vacation leave. We will use an AWS Lambda function to define the logic that checks for the available vacation days and book new ones.

For this example, we will generate some employee data using an [SQLite](https://www.sqlite.org/) database

## Pre-requisites
Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [2]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

Let's now check the boto3 version to ensure the correct version has been installed. Your version should be bigger or equal to 1.34.90.

In [3]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.34.90


In [4]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

Let's now create the boto3 clients for the required AWS services

In [5]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

Next we can set some configuration variables for the agent and for the lambda function being created

In [6]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '350685477109')

In [7]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for providing HR assistance to book holidays"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vactions days for an employee and book new vacations in the system"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## Creating Lambda Function

We will now create a lambda function that interacts with the SQLite file `employee_database.db`. To do so we will:
1. Create the `employee_database.db` file which contains the employee database with some generated data.
2. Create the `lambda_function.py` file which contains the logic for our lambda function
3. Create the IAM role for our Lambda function
4. Create the lambda function infrastructure with the required permissions

In [8]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year - 3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y-%m-%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

Let's now create our lambda functio. It implements the functionality for `get_available_vacations_days` for a given employee_id and `book_vacations` for an employee giving a start and end date

In [9]:
%%writefile lambda_function.py
import json
import sqlite3

def get_available_vacations_days(employee_id):
    # Connect to the SQLite database
    conn = sqlite3.connect('employee_database.db')
    c = conn.cursor()

    if employee_id:

        # Fetch the available vacation days for the employee
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]  # Unpack the tuple
            print(f"Available vacation days for employed_id {employee_id}: {available_vacation_days}")
            return available_vacation_days
        else:
            print(f"No vacation data found for employed_id {employee_id}")
    else:
        raise Exception(f"No employeed id provided")

    # Close the database connection
    conn.close()
    
    
def book_vacation(employee_id, start_date, end_date):
    # Connect to the SQLite database
    conn = sqlite3.connect('employee_database.db')
    c = conn.cursor()
    try:
        # Calculate the number of vacation days
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        vacation_days = (end_date - start_date).days + 1

        # Get the current year
        current_year = start_date.year

        # Check if the employee exists
        c.execute("SELECT * FROM employees WHERE employee_id = ?", (employee_id,))
        employee = c.fetchone()
        if employee is None:
            print(f"Employee with ID {employee_id} does not exist.")
            return

        # Check if the vacation days are available for the employee in the current year
        c.execute("SELECT employee_vacation_days_available FROM vacations WHERE employee_id = ? AND year = ?", (employee_id, current_year))
        available_days = c.fetchone()
        if available_days is None or available_days[0] < vacation_days:
            print(f"Employee with ID {employee_id} does not have enough vacation days available for the requested period.")
            return

        # Insert the new vacation into the planned_vacations table
        c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, vacation_days))

        # Update the vacations table with the new vacation days taken
        c.execute("UPDATE vacations SET employee_vacation_days_taken = employee_vacation_days_taken + ?, employee_vacation_days_available = employee_vacation_days_available - ? WHERE employee_id = ? AND year = ?", (vacation_days, vacation_days, employee_id, current_year))

        conn.commit()
        print(f"Vacation booked successfully for employee with ID {employee_id} from {start_date} to {end_date}.")
        # Close the database connection
        conn.close()
        return f"Vacation booked successfully for employee with ID {employee_id} from {start_date} to {end_date}."
    except Exception as e:
        raise Exception(f"Error occurred: {e}")
        conn.rollback()
        # Close the database connection
        conn.close()
        return f"Error occurred: {e}"
        

def lambda_handler(event, context):
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }

    if function == 'get_available_vacations_days':
        employee_id = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]

        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        vacation_days = get_available_vacations_days(employee_id)
        responseBody =  {
            'TEXT': {
                "body": f"available vacation days for employed_id {employee_id}: {vacation_days}"
            }
        }
    elif function == 'book_vacation':
        employee_id = None
        start_date = None
        end_date = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]
            
        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        if not start_date:
            raise Exception("Missing mandatory parameter: start_date")
        if not end_date:
            raise Exception("Missing mandatory parameter: end_date")
        
        completion_message = book_vacation(employee_id, start_date, end_date)
        responseBody =  {
            'TEXT': {
                "body": completion_message
            }
        }  
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }

    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response


Writing lambda_function.py


Next let's create the lambda IAM role and policy to invoke a Bedrock model

In [10]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '2c2eb681-fb36-48d3-b02d-caf488709055',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 24 Apr 2024 16:18:11 GMT',
   'x-amzn-requestid': '2c2eb681-fb36-48d3-b02d-caf488709055',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

We can now package the lambda function to a Zip file and create the lambda infrastructure using boto3

In [11]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation and the agent IAM role with the policy associated to it. We will allow this agent to invoke the Claude Sonnet model

In [12]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [13]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '872dca2e-f387-45d3-8af7-d5489fc5048f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 24 Apr 2024 16:18:22 GMT',
   'x-amzn-requestid': '872dca2e-f387-45d3-8af7-d5489fc5048f',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### Creating agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underline foundation model and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [14]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': '227e0cb8-a41e-4ae7-983f-2f23b2b521e4',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 24 Apr 2024 16:18:23 GMT',
   'content-type': 'application/json',
   'content-length': '639',
   'connection': 'keep-alive',
   'x-amzn-requestid': '227e0cb8-a41e-4ae7-983f-2f23b2b521e4',
   'x-amz-apigw-id': 'WvSQaETVPHcEu_A=',
   'x-amzn-trace-id': 'Root=1-662930cf-0b1854df52b6f44e5e910e05'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:350685477109:agent/9NBG3GJOJN',
  'agentId': '9NBG3GJOJN',
  'agentName': 'hr-assistant-function-def2',
  'agentResourceRoleArn': 'arn:aws:iam::350685477109:role/AmazonBedrockExecutionRoleForAgents_hr-assistant-function-def2',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 4, 24, 16, 18, 23, 228021, tzinfo=tzlocal()),
  'description': 'Agent for providing HR assistance to book holidays',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeco

Let's now store the agent id in a local variable to use it on the next steps

In [15]:
agent_id = response['agent']['agentId']
agent_id

'9NBG3GJOJN'

## Create Agent Action Group
We will now create an agent action group that uses the lambda function created before. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In this example, we will provide the Action Group functionality using a `functionSchema`. You can also provide and `APISchema`. The notebook [02-create-agent-with-api-schema.ipynb](02-create-agent-with-api-schema/02-create-agent-with-api-schema.ipynb) provides an example of it.

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [16]:
agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'book_vacations',
        'description': 'book the vacation days for the employeed',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            },
            "start_vacation_period": {
                "description": "the start date for the vacation booking",
                "required": True,
                "type": "string"
            },
            "end_vacation_period": {
                "description": "the end date for the vacation booking",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [17]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [18]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'd982e95d-fd45-40db-8396-14c092571f1e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 24 Apr 2024 16:18:53 GMT',
   'content-type': 'application/json',
   'content-length': '1241',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd982e95d-fd45-40db-8396-14c092571f1e',
   'x-amz-apigw-id': 'WvSVKGoSPHcELOg=',
   'x-amzn-trace-id': 'Root=1-662930ed-1f274fec3d2164bf2ac4edac'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:350685477109:function:hr-assistant-function-def2-us-west-2-350685477109'},
  'actionGroupId': 'VFDBADW8Y1',
  'actionGroupName': 'VacationsActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': '9NBG3GJOJN',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 4, 24, 16, 18, 53, 710999, tzinfo=tzlocal()),
  'description': 'Actions for getting the number of available vactions days for an employee and book new vacations in the system',
  'functio

## Allowing Agent to invoke Action Group Lambda
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [19]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [20]:
response

{'ResponseMetadata': {'RequestId': 'd713cf5d-3ea1-4ce9-87b0-b9e12fea6985',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Wed, 24 Apr 2024 16:18:53 GMT',
   'content-type': 'application/json',
   'content-length': '377',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd713cf5d-3ea1-4ce9-87b0-b9e12fea6985'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-west-2:350685477109:function:hr-assistant-function-def2-us-west-2-350685477109","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-west-2:350685477109:agent/9NBG3GJOJN"}}}'}

## Preparing Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [21]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': 'af87e13f-4ddd-46ca-8636-a5da5df13e0d', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 24 Apr 2024 16:18:54 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': 'af87e13f-4ddd-46ca-8636-a5da5df13e0d', 'x-amz-apigw-id': 'WvSVPFrzvHcEN7Q=', 'x-amzn-trace-id': 'Root=1-662930ee-562a4eee0365f1852b43ac9e'}, 'RetryAttempts': 0}, 'agentId': '9NBG3GJOJN', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 4, 24, 16, 18, 54, 136254, tzinfo=tzlocal())}


## Invoke Agent

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [22]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False
# Pause to make sure agent alias is ready
# time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Who many available vacation days does employee_id 1 has?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-04-24 16:19:24,241] p2600 {3948257912.py:24} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 24 Apr 2024 16:19:24 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '6a1375a0-0256-11ef-a82d-061e18f5d56f',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '2232c68b-a931-4b81-8f69-9e5c7cbd94b2'},
                      'HTTPStatusCode': 200,
                      'RequestId': '2232c68b-a931-4b81-8f69-9e5c7cbd94b2',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fcf07b41f60>,
 'contentType': 'application/json',
 'sessionId': '6a1375a0-0256-11ef-a82d-061e18f5d56f'}


In [23]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-04-24 16:19:24,395] p2600 {<timed exec>:11} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "9NBG3GJOJN",
  "agentVersion": "DRAFT",
  "sessionId": "6a1375a0-0256-11ef-a82d-061e18f5d56f",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "</invoke>",
            "</answer>",
            "</error>"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\" You are an HR agent, helping employees understand HR policies and manage vacation time You have been provided with a set of functions to answer the user's question. You must call the functions in the format below: <function_calls> <invoke> <tool_name>$TOOL_NAME<\\/tool_name> <parameters> <$PARAMETER_NAME>$PARAMETER_VALUE<\\/$PARAMETER_NAME> ... <\\/parameters> <\\/invoke> <\\/function_calls> Here are the functions availab

CPU times: user 14 ms, sys: 5.74 ms, total: 19.7 ms
Wall time: 7.29 s


In [24]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

Employee with id 1 has 1 available vacation day.


## Clean up (optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. update the action group to disable it
2. delete agent action group
4. delete agent
5. delete lambda function
6. delete the created IAM roles and policies


In [27]:
# This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [28]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [29]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

{'ResponseMetadata': {'RequestId': '0f758618-a004-4dd9-9c38-f05968b1ce50',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Wed, 24 Apr 2024 16:17:30 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0f758618-a004-4dd9-9c38-f05968b1ce50'},
  'RetryAttempts': 1}}

In [30]:
# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)


## Conclusion
We have now experimented with using boto3 SDK to create, invoke and delete an agent created using function definitions.

## Take aways
Adapt this notebook to create new agents using function definitions for your application

## Thank You